- Setup the environment

In [1]:
import sys
import os
cur_dir = os.getcwd()
# Add the current directory to system path (including parent folder)
sys.path.append(cur_dir)
sys.path.append(os.path.split(cur_dir)[0])

- Load input data

Create sample data

In [2]:
import pandas as pd
import numpy as np
# Prepare sample data
timestamp = pd.date_range(start='2020-01-01', periods=300, freq='D')
time_series = pd.DataFrame({'values': np.random.randn(300)}, index=timestamp)
labels = pd.DataFrame({'label': np.random.randint(0, 3, size=300)}, index=timestamp)
ext_features = pd.DataFrame({
    'feature1': np.random.randn(300),
    'feature2': np.random.randn(300)
}, index=timestamp)

combined_data = time_series.join(labels).join(ext_features)

Train / Validation / Test split

In [3]:
# Train / Validation / Test Split
train_data = combined_data[:100]
val_data = combined_data[100:200]
test_data = combined_data[200:]

# Read targets
train_target = train_data.label.values
val_target = val_data.label.values
test_target = test_data.label.values

# Read features
train_features = train_data[['feature1', 'feature2']].values
val_features = val_data[['feature1', 'feature2']].values
test_features = test_data[['feature1', 'feature2']].values

In [4]:
from pl_model_utils import TimeSeriesDataModule
# Instantiate data module and model
data_module = TimeSeriesDataModule(
    train_target, train_features,
    val_target, val_features,
    test_target, test_features,
    batch_size=16
)

# Setup the data for model
data_module.setup()

c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Classification with Basic Autoencoder

- Compile simple Autoencoder for Classification

In [5]:
from pl_autoencoder_classifiers import AutoencoderClassifier
ae_model = AutoencoderClassifier(context_length=1, num_classes=3, num_features=2)

In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint
# Define a checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    mode='min',
    save_top_k=1,
    save_last=True,
)

- Train the Autoencoder

In [7]:
from pytorch_lightning import Trainer
# Train the model
trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback])
trainer.fit(ae_model, data_module.train_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | encoder    | Sequential | 8.8 K 
1 | decoder    | Sequential | 8.4 K 
2 | classifier | Sequential | 2.2 K 
3 | softmax    | Softmax    | 0     
------------------------------------------
19.4 K    Trainable params
0         Non-trainable params
19.4 K    Total params
0.078     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\jedre\miniconda3\envs\GluonTS\lib\si

Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 102.87it/s, v_num=93]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 7/7 [00:00<00:00, 87.55it/s, v_num=93] 


- Cross-Validation (Using Combinatorial Purged K-Fold CV)

In [17]:
from pl_model_utils import cross_validate_model
from CPCV.cpcv import CombPurgedKFoldCVLocal


pred_times = pd.Series(val_data.index, index=val_data.index)
eval_times = pd.Series(val_data.index, index=val_data.index)

# Construct CPCV in-line with DePrado method
cpcv = CombPurgedKFoldCVLocal(
    n_splits=10,
    n_test_splits=1,
    embargo_td=pd.Timedelta(days=2)
)

cv_split = cpcv.split(
    pd.DataFrame(data_module.val_features, index=val_data.index), 
    pd.Series(data_module.val_target, index=val_data.index), 
    pred_times, 
    eval_times)

# Perform cross-validation
cv_results = cross_validate_model(
    X=data_module.val_features,
    y=data_module.val_target,
    model=ae_model,
    cv_split=cv_split)

0it [00:00, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | encoder    | Sequential | 8.8 K 
1 | decoder    | Sequential | 8.4 K 
2 | classifier | Sequential | 2.2 K 
3 | softmax    | Softmax    | 0     
------------------------------------------
19.4 K    Trainable params
0         Non-trainable params
19.4 K    Total params
0.078     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\jedre\miniconda3\e

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 67.44it/s, v_num=94] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 57.01it/s, v_num=94]


1it [00:04,  4.51s/it]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params
------------------------------------------
0 | encoder    | Sequential | 8.8 K 
1 | decoder    | Sequential | 8.4 K 
2 | classifier | Sequential | 2.2 K 
3 | softmax    | Softmax    | 0     
------------------------------------------
19.4 K    Trainable params
0         Non-trainable params
19.4 K    Total params
0.078     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\jedre\minicond

...

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 80.44it/s, v_num=103]

10it [00:41,  4.14s/it]

10it [00:41,  4.18s/it]

Cross-Validation Classification Report:
           0.0  1.0  2.0  accuracy  macro avg  weighted avg
precision  1.0  1.0  1.0       1.0        1.0           1.0
recall     1.0  1.0  1.0       1.0        1.0           1.0
f1-score   1.0  1.0  1.0       1.0        1.0           1.0
support    3.6  3.7  2.7       1.0       10.0          10.0


In [19]:
from mc_dropout import mc_dropout # final, one-line function to implement mc dropout (As suggested here: https://github.com/Lightning-AI/pytorch-lightning/blob/f35e2210e240b443fd4dafed8fe2e30ee7d579ea/docs/source/common/production_inference.rst#prediction-api)
predictions_mean, predictions_std = mc_dropout(ae_model, data_module.val_dataloader(), mc_iterations=100)

100%|██████████| 7/7 [00:00<00:00, 11.63it/s]


In [20]:
np.argmax(predictions_mean, axis=1)

array([1, 1, 2, 1, 2, 0, 2, 2, 1, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 2, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 1, 2, 0, 0,
       0, 2, 0, 0, 1, 1, 1, 1, 2, 0, 1, 0, 1, 0, 2, 0, 0, 2, 2, 1, 1, 0,
       2, 2, 2, 1, 0, 1, 2, 1, 2, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 2, 2, 0,
       2, 1, 0, 2, 2, 2, 1, 2, 1, 1, 1, 2], dtype=int64)

## Classification with Attention-Enchanced Autoencoder

## Hyperparameter Tuning

In [22]:
from pytorch_lightning.callbacks import EarlyStopping
from pl_autoencoder_classifiers import AutoencoderAttentionClassifier
from pl_model_utils import TimeSeriesDataset
import pytorch_lightning as pl
import torch
from sklearn.model_selection import TimeSeriesSplit
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import optuna
import numpy as np

def ae_attention_objective(trial):
    context_length = 1
    num_classes = 3
    num_features = 2

    # Suggest hyperparameters
    lr = trial.suggest_categorical('lr', [1e-5, 1e-3, 1e-2])
    num_heads = trial.suggest_categorical('num_heads', [1, 2, 4])
    dropout_prob = trial.suggest_categorical('dropout_prob', [0.1, 0.3, 0.5])
    hidden_units = trial.suggest_categorical('hidden_units', [64, 128, 256])
    embed_dim = trial.suggest_categorical('embed_dim', [32, 64, 128])
    classifier_units = trial.suggest_categorical('classifier_units', [16, 32, 64])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Initialize the model with suggested hyperparameters
    model = AutoencoderAttentionClassifier(
        context_length=context_length,
        num_classes=num_classes,
        num_features=num_features,
        num_heads=num_heads,
        dropout_prob=dropout_prob,
        hidden_units=hidden_units,
        embed_dim=embed_dim,
        classifier_units=classifier_units,
        lr=lr
    )

    # Assuming you have your dataset in `X` and `y`
    X, y = data_module.val_features, data_module.val_target
    # X = np.array(X_train)  # Ensure X_train is a NumPy array
    # y = np.array(y_train)  # Ensure y_train is a NumPy array

    # Time series split
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []

    for train_index, val_index in tscv.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Create DataLoader for the training and validation fold
        train_dataset = TimeSeriesDataset(
            torch.tensor(y_train_fold, dtype=torch.float32), 
            torch.tensor(X_train_fold, dtype=torch.float32)
            )
        
        val_dataset = TimeSeriesDataset(
            torch.tensor(y_val_fold, dtype=torch.float32), 
            torch.tensor(X_val_fold, dtype=torch.float32)
            )
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Initialize trainer
        trainer = pl.Trainer(
            max_epochs=10,
            callbacks=[EarlyStopping(monitor='train_loss', patience=3, mode='min')],
            logger=False,
            enable_checkpointing=False
        )

        # Train the model
        trainer.fit(model, train_loader)

        # Validate the model

        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for batch in val_loader:
                targets, features = batch
                _, classification = model(targets, features)
                # preds = torch.argmax(classification, dim=1)
                all_preds.extend(classification.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        # val_predictions = trainer.predict(model, val_loader)
        # val_predictions = torch.cat([x for x in val_predictions], dim=0).numpy()
        
        val_loss = log_loss(all_targets, all_preds)
        cv_scores.append(val_loss)

    return np.mean(cv_scores)

In [ ]:
# Run the Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(ae_attention_objective, n_trials=10)

In [ ]:
# Print best hyperparameters
print("Best hyperparameters:", study.best_params)

In [ ]:
ae_attention_model = AutoencoderAttentionClassifier(
    context_length=1, 
    num_classes=3, 
    num_features=2,
    lr=study.best_params['lr'],
    num_heads=study.best_params['num_heads'],
    dropout_prob=study.best_params['dropout_prob'],
    hidden_units=study.best_params['hidden_units'],
    embed_dim=study.best_params['embed_dim'],
    classifier_units=study.best_params['classifier_units']
    )

In [23]:
ae_attention_model = AutoencoderAttentionClassifier(
    context_length=1, 
    num_classes=3, 
    num_features=2,
    lr=0.001,
    num_heads=1,
    dropout_prob=0.1,
    hidden_units=256,
    embed_dim=128,
    classifier_units=64
    )

In [24]:
# Train the model
trainer = Trainer(max_epochs=10, callbacks=[checkpoint_callback])
trainer.fit(ae_attention_model, data_module.train_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory c:\Users\jedre\OneDrive\Pulpit\State_Street\Programs\Python\ML_Research_Project_2024\Autoencoder\lightning_logs\version_93\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total es

Epoch 9: 100%|██████████| 7/7 [00:00<00:00, 65.29it/s, v_num=104]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 7/7 [00:00<00:00, 55.69it/s, v_num=104]


In [27]:
# Construct CPCV in-line with DePrado method
cpcv = CombPurgedKFoldCVLocal(
    n_splits=10,
    n_test_splits=1,
    embargo_td=pd.Timedelta(days=2)
)

cv_split = cpcv.split(
    pd.DataFrame(data_module.val_features, index=val_data.index), 
    pd.Series(data_module.val_target, index=val_data.index), 
    pred_times, 
    eval_times)


# Perform cross-validation
cv_results = cross_validate_model(
    X=data_module.val_features, 
    y=data_module.val_target, 
    model=ae_attention_model,
    cv_split=cv_split)

0it [00:00, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this ma

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 33.99it/s, v_num=105]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 27.38it/s, v_num=105]


1it [00:09,  9.21s/it]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | encoder    | Sequential         | 33.9 K
1 | attention  | MultiheadAttention | 66.0 K
2 | decoder    | Sequential         | 33.3 K
3 | classifier | Sequential         | 8.5 K 
4 | softmax    | Softmax            | 0     
--------------------------------------------------
141 K     Trainable params
0         Non-trainable params
141 K     Total params
0.567     Total estimated model params size (MB)
c:\Users\jedre\miniconda3\envs\GluonTS\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on thi

...

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 45.10it/s, v_num=114]

10it [01:13,  7.05s/it]

10it [01:13,  7.39s/it]

Cross-Validation Classification Report:
           0.0       1.0  2.0  accuracy  macro avg  weighted avg
precision  1.0  0.980000  0.9      0.99   0.960000      0.982000
recall     1.0  1.000000  0.9      0.99   0.966667      0.990000
f1-score   1.0  0.988889  0.9      0.99   0.962963      0.985556
support    3.6  3.700000  2.7      0.99  10.000000     10.000000


In [28]:
print(cv_results)

           0.0       1.0  2.0  accuracy  macro avg  weighted avg
precision  1.0  0.980000  0.9      0.99   0.960000      0.982000
recall     1.0  1.000000  0.9      0.99   0.966667      0.990000
f1-score   1.0  0.988889  0.9      0.99   0.962963      0.985556
support    3.6  3.700000  2.7      0.99  10.000000     10.000000


In [29]:
predictions_mean, predictions_std = mc_dropout(ae_attention_model, data_module.val_dataloader(), mc_iterations=100)

100%|██████████| 7/7 [00:01<00:00,  3.77it/s]


In [30]:
predicted_labels = np.argmax(predictions_mean, axis=1)

In [31]:
# Example output with probabilities and uncertainty
for i, (mean, std) in enumerate(zip(predictions_mean, predictions_std)):
    # softmax_probs = np.exp(mean) / np.sum(np.exp(mean)) # Softmax to get probabilities
    print(f'Sample {i}: Predicted Label = {predicted_labels[i]}, Probabilities = {mean}, Uncertainty (std) = {std}')

Sample 0: Predicted Label = 1, Probabilities = [0.03628777 0.9482291  0.01548309], Uncertainty (std) = [0.10941979 0.15611643 0.04675435]
Sample 1: Predicted Label = 1, Probabilities = [0.05393438 0.9226053  0.0234603 ], Uncertainty (std) = [0.12905784 0.1852064  0.0562723 ]
Sample 2: Predicted Label = 2, Probabilities = [0.03300069 0.04342121 0.923578  ], Uncertainty (std) = [0.10550195 0.13881221 0.24423368]
Sample 3: Predicted Label = 1, Probabilities = [0.0360552  0.94780135 0.01614348], Uncertainty (std) = [0.10876472 0.15743658 0.04884774]
Sample 4: Predicted Label = 2, Probabilities = [0.03975081 0.05318822 0.907061  ], Uncertainty (std) = [0.11368055 0.15210102 0.26571855]
Sample 5: Predicted Label = 0, Probabilities = [0.9421     0.04370263 0.01419731], Uncertainty (std) = [0.18504454 0.13968307 0.04548705]
Sample 6: Predicted Label = 2, Probabilities = [0.02847718 0.03841257 0.9331102 ], Uncertainty (std) = [0.09709278 0.13094921 0.22801423]
Sample 7: Predicted Label = 2, Pro

- Export the reults

In [32]:
import pandas as pd
# Save test predictions to a CSV
test_df = pd.DataFrame({
    'Prediction': predicted_labels,
    'Probability_0': [p[0] for p in predictions_mean],
    'Probability_1': [p[1] for p in predictions_mean],
    'Probability_2': [p[2] for p in predictions_mean],  # Adjust based on num_classes
    'Uncertainty_0': [u[0] for u in predictions_std],
    'Uncertainty_1': [u[1] for u in predictions_std],
    'Uncertainty_2': [u[2] for u in predictions_std] 
})

test_df.to_csv('test_predictions.csv', index=False)

- Export the model

In [33]:
from torch import save
save(ae_attention_model.state_dict(), 'autoencoder_attention_classifier.pth')

In [34]:
import h5py

with h5py.File('ae_data.h5', 'w') as f:
    f.create_dataset('X', data=data_module.val_features)
    f.create_dataset('y', data=data_module.val_target)

## Explainable AI Part

In [ ]:
import sys
import os
sys.path.append(os.getcwd())

In [ ]:
import h5py

with h5py.File('ae_data.h5', 'r') as f:
    X = f['X'][:]
    y = f['y'][:]

In [ ]:
from pl_autoencoder_classifiers import AutoencoderAttentionClassifier
from pl_model_utils import TimeSeriesDataset
from torch.utils.data import DataLoader

In [ ]:
val_dataset = TimeSeriesDataset(
            torch.tensor(y, dtype=torch.float32), 
            torch.tensor(X, dtype=torch.float32)
            )

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
batch = next(iter(val_loader))
targets, features = batch

In [ ]:
ae_model = AutoencoderAttentionClassifier(
    context_length=1, 
    num_classes=3, 
    num_features=2,
    lr=0.001,
    num_heads=1,
    dropout_prob=0.1,
    hidden_units=256,
    embed_dim=128,
    classifier_units=64)
ae_model.load_state_dict(torch.load('autoencoder_attention_classifier.pth'))
ae_model.eval()

In [ ]:
import shap

e = shap.DeepExplainer(ae_model, features)

In [ ]:
from transformers import modeling_tf_utils